In [1]:
from collections import defaultdict, namedtuple
import numpy as np
import pandas as pd
import math
import random

In [2]:
import duolingo_replica as dr
import duolingo_original as do

We load the data and select a subset of 5000 observations to test the models.

In [1]:
data   = pd.read_csv('learning_traces.13m_en_10.csv' )
subset = data.sample( n = 100000, random_state = 2023 )

NameError: name 'pd' is not defined

# 1. LOGIT

## 1.1. Replication

We perform the train-test split and, in addition, obtain the list of predictor variables. The predictor variables include: 'right,' 'wrong,' 'bias,', 'time' and dummy variables for lexemes.

In [ ]:
%%time
trainset, testset, feature_vars = dr.read_data( subset, method = 'lr', omit_lexemes = True )

We define and fit a Logistic Regression Model for the replication code

In [ ]:
%%time

model_1 = dr.logit_model( feature_columns = feature_vars )
model_1.train( trainset )

We evaluate the model with the test set and obtain results. The h_seed allows replicating the values of h_hat, which are random for logistic regression

In [2]:
model_1.test_model( testset, h_seed = 5 )

NameError: name 'model_1' is not defined

In [3]:
%%time
trainset, testset, feature_vars = dr2.read_data( subset, method = 'lr', omit_lexemes = True )

NameError: name 'dr2' is not defined

In [33]:
%%time

model_2 = dr2.logit_model( feature_columns = feature_vars )
model_2.train( trainset )

CPU times: total: 344 ms
Wall time: 347 ms


In [34]:
model_2.test_model( testset, h_seed = 5 )

-----------------------------
            Results          
-----------------------------
Total Loss : 383916.093
p          : 126.356
h          : 383789.734
l2         : 0.003
mae (p)    : 0.316
cor (p)    : -0.085
mae (h)    : 151.389
cor (h)    : 0.030
-----------------------------


We export theta values for the Logit replication model

In [24]:
model_1.dump_theta( 'logit_replication_thetas_old.txt' )

In [25]:
model_2.dump_theta( 'logit_replication_thetas_new.txt' )

## 1.2. Original code

We perform the train-test split and, in addition, obtain the list of predictor variables. The predictor variables include: 'right,' 'wrong,' 'bias,' and dummy variables for lexemes.

In [26]:
trainset2, testset2 = do.read_data( subset, method = 'lr' )

done!


We define and fit a Logistic Regression Model for the original code

In [27]:
%%time

model_2 = do.SpacedRepetitionModel( method = 'lr' )
model_2.train( trainset2 )

CPU times: total: 125 ms
Wall time: 121 ms


We evaluate the model with the test set and obtain results. The first value on the right corresponds to the metric 'Total Loss.' The other metrics are named accordingly. The metrics are almost exactly the same as in the replication code.

In [28]:
model_2.eval( testset2, h_seed = 5 )

383915.918 (p=126.181, h=383789.734, l2=0.003)	mae(p)=0.316	cor(p)=-0.085	mae(h)=151.389	cor(h)=0.030


We export theta values for the Logit original model. Thetas are almost exactly the same as in the replication code.

In [11]:
model_2.dump_weights( 'logit_original_thetas.txt' )

# 2. HLR Model

## 2.1. Replication

We perform the train-test split and, in addition, obtain the list of predictor variables. The predictor variables include: 'right,' 'wrong,' 'bias,' and dummy variables for lexemes. Unlike Model 1, in this case, the variable 'time' is not included as a predictor.

In [39]:
%%time

trainset3, testset3, feature_vars3 = dr2.read_data( subset, method = 'hlr', omit_lexemes = False )

CPU times: total: 8.53 s
Wall time: 8.55 s


We train the HLR replication model

In [40]:
%%time

model_3 = dr.HLR_model( feature_columns = feature_vars3, omit_h_term = False )
model_3.train( trainset3 )

CPU times: total: 34min 14s
Wall time: 34min 40s


In [41]:
%%time

model_4 = dr2.HLR_model( feature_columns = feature_vars3, omit_h_term = False )
model_4.train( trainset3 )

CPU times: total: 11.9 s
Wall time: 12 s


We evaluate the model with the test set and obtain results

In [42]:
%%time

model_3.test_model( testset3 )

-----------------------------
            Results          
-----------------------------
Total Loss : 3602154.522
p          : 2660.700
h          : 3599493.770
l2         : 0.052
mae (p)    : 0.345
cor (p)    : 0.052
mae (h)    : 145.137
cor (h)    : -0.029
-----------------------------
CPU times: total: 2min 20s
Wall time: 2min 23s


In [43]:
%%time

model_4.test_model( testset3 )

-----------------------------
            Results          
-----------------------------
Total Loss : 3602154.522
p          : 2660.700
h          : 3599493.770
l2         : 0.052
mae (p)    : 0.345
cor (p)    : 0.052
mae (h)    : 145.137
cor (h)    : -0.029
-----------------------------
CPU times: total: 359 ms
Wall time: 225 ms


We export theta values for the Logit replication model

In [14]:
model_3.dump_theta( 'hlr_replication_thetas_old.txt' )

In [8]:
model_4.dump_theta( 'hlr_replication_thetas_new.txt' )

## 2.2. Original

We perform the train-test split and, in addition, obtain the list of predictor variables. The predictor variables include: 'right,' 'wrong,' 'bias,' and dummy variables for lexemes. Unlike Model 3, in this case, the variable 'time' is not included as a predictor.

In [43]:
trainset2, testset2 = do.read_data( subset, method = 'hlr' )

0...done!


We fit the HLR original model. We omit h_term.

In [44]:
%%time

model_4 = do.SpacedRepetitionModel( method = 'hlr', omit_h_term = False )
model_4.train( trainset2 )

CPU times: total: 1.41 s
Wall time: 1.43 s


We evaluate the model with the test set and obtain results. The first value on the right corresponds to the metric 'Total Loss.' The other metrics are named accordingly. The metrics are almost exactly the same as in the replication code.

In [45]:
model_4.eval( testset2 )

3598756.936 (p=2620.455, h=3596136.357, l2=0.124)	mae(p)=0.342	cor(p)=0.052	mae(h)=145.049	cor(h)=-0.030


We export theta values for the HLR original model. Thetas are almost exactly the same as in the replication code.

In [19]:
model_4.dump_weights( 'hlr_original_thetas.txt' )